# Grappa Student Seminar
### Weekly programming assignments - Week 1
#### Dylan van Arneman, Gijs Leguijt, Sven Poelmann, Yoran Yeh
##### June 2019

[Github link](https://github.com/adam-coogan/GRAPPA_Student_Seminar_2019)

Support $\LaTeX$

## Imports

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.constants import *
from scipy.optimize import root
#matplotlib.rcParams['text.usetex'] = True
#matplotlib.rcParams['text.latex.unicode'] = True
%matplotlib inline

## Units & Constants

We used natural units, $c = \hbar = k_B = 1$, the following constants are used to convert units.

In [ ]:
"""Units________________________________________________________________________________________________________________"""
eV      = 1                                                                                                  #Electron volt
J       = eV / e                                                                                                     #Joule
s       = 1 / (hbar * J)                                                                                            #Second
m       = s / c                                                                                                      #Meter
kg      = J * c**(2)                                                                                             #Kilograms
K       = J * k                                                                                                     #Kelvin

"""Constants____________________________________________________________________________________________________________""" 

G       = value("Newtonian constant of gravitation") * (m**3 / (kg * s**2))                                #Newton constant
M_pl    = value('Planck mass') * kg                                                                            #Planck mass
GeV     = 1.0e9 * eV                                                                                    #Giga electron volt
MeV     = 1.0e6 * eV                                                                                    #Mega electron volt
G_f     = value('Fermi coupling constant') * GeV**-2                                                        #Fermi constant
T_CMB   = 2.725 * K                                                                                        #CMB temperature
Mpc     = 3.085e22 * m                                                                                         #Mega parsec
cm      = m / 100                                                                                               #Centimeter
H       = 67.8e3 * (m / s) / Mpc                                                                           #Hubble constant
h_hub   = H / (1.0e5 * (m / s) / Mpc)                                                              #Reduced hubble constant
rho_c   = 1.9e-26 * h_hub**2 * kg / m**3                                                                  #Critical density
zheta_3 = 1.202                                                                              #Zheta funtion with argument 3

## Main exercise and code

In this notebook we determine the thermal freeze-out temperature of dark matter particles. We do this for two different DM cases, namely the hot dark matter (relativistic) and the cold dark matter (non-relativistic) cases. Next, we use these freeze-out temperatures to determine the current dark matter abundance.

## Part 1: Instantaneous freeze-out
### Hot Dark Matter

In the hot dark matter (HDM) case we have the following particle number density:

$n(T_{fo}) = \frac{\zeta(3)}{\pi^2} g^\star T_{fo}^3$, where $g^\star$ is the number of degrees of freedom of the DM particle. Note that this equation only holds for equilibrium. To calculate the DM abundance today, we have to redshift this particle density to the current time.

Use $n(T_0) = n(T_{fo}) \frac{s_0}{s_{fo}}$. We take $ \frac{s_0}{s_{fo}} = \frac{g_s^\star(T_0) \cdot (T_0)^3}{g_s^\star(T_{fo}) \cdot (T_{fo})^3} \xrightarrow{} n(T_0) = n(T_{fo}) \frac{g_s^\star(T_0)\cdot (T_0)^3}{g_s^\star(T_{fo})\cdot(T_{fo})^3}$. Here, $g_s^\star$ is related to the entropy of the plasma and should not be confused with the degrees of freedom of the DM particle.

Now to find $T_{fo}$: use the Friedmann equation and the freeze-out condition to find the freeze-out temperature. 

The freeze-out condition:
$n(T_{fo}) \sigma = H =  \left(\frac{8 \pi G}{3}\right)^3 \rho^{1/2}$. During radiation domination the energy density is $\rho = \frac{\pi^2}{30} g^\star(T) T^4$. If we fill this in and solve for the temperature (analytically), we find: 

$ T_{fo}^3 = \sqrt{\frac{4}{45} \frac{G \pi^7}{g^\star(T_{fo})}} \frac{1}{\zeta(3) G_F^2}$.

Now we fill in this expression for $T_{fo}$ into the expression for $n(T_{fo})$, and redshift this accordingly. 

Finally, to determine the abundance we use $ \Omega_{HDM} = \frac{m n(T_0)}{\rho_{crit}}$, and fill in $n(T_0)$ as determined above. Where we model sigma to be $\sigma = G_F^2 T_{fo}^2$.

N.B.: when we did the math it turned out $T_{fo}$ dropped out, however to keep the method between cold and hot dark matter consistent, we calculated $T_{fo}$ anyway.

We plot the abundance as a function of the DM particle mass and for different cross sections, as seen in the figure below.

In [ ]:
#More constants
g_s_T0 = 3.94                                                                                      #g^\star at current time 
g_s_Tfo = 106.75                                                                                    #g^\start at freeze out
g_dof = 2                                                                                               #degrees of freedom

In [ ]:
#Functions needed
def fo(g_dof,G_f):
    """Calculates freeze out temperature
    Input: 2 single values, output: single value"""
    T_fo = (np.sqrt((4.0/45) * (G*(np.pi**7) /(g_dof))) * 1/(zheta_3 * G_f**2))**(1.0/3)
    return T_fo

def nd_hdm(g_dof,T_fo):
    """Calculates number density at freeze out temperature
    Input: 2 single values, output: single value"""
    n_hdm = (zheta_3* g_dof*T_fo**3 )/np.pi**2
    return n_hdm

def abun(n_fo,mass):
    """Calculates abundancy
    Input: 2 single values, output: single value"""
    n_0       = n_fo*(g_s_T0 * T_CMB**3)/(g_s_Tfo * T_fo**3)                                 #Number density at current time
    Omega_hdm = mass * n_0 / rho_c
    return Omega_hdm

In [ ]:
#To test the code in 1 particular case
mass = np.linspace(1.0e-2,1.0e2,500)                                                               #Sets a linspace for mass
T_fo = fo(g_dof,G_f)                                                                                 #Calculates T_freezeout
n_fo = nd_hdm(g_dof,T_fo)                                                                         #Calculates number density
omega_hdm = abun(n_fo,mass)                                                                        #Calculates the abundancy

plt.figure(1,figsize=(14, 6))                                  #Plots the HDM-abundance as function of mass on semilog scale
plt.semilogx(mass,omega_hdm,label = r'$\Omega_{HDM}$')
plt.ylabel(r'$\Omega_{HDM}$ ',fontsize=18)
plt.xlabel(r'Mass [$eV$]',fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'$\Omega_{HDM}$ for Hot Dark Matter',fontsize=20)
plt.grid(True)

plt.legend(prop=dict(size=18))
plt.show()

In [ ]:
#HDM abundance as function of mass for different cross-section
plt.figure(1,figsize=(14, 6))
factor = np.logspace(-1,2,4)                                  #used to vary the Fermi's constant and thus the cross-section 
mass   = np.linspace(1.0e-2,1.0e2,500)

for i in factor:
    T_fo      = fo(g_dof,i*G_f)                                                                     #Freeze out temperature
    n_fo      = nd_hdm(g_dof,T_fo)                                                            #Number density at freeze out
    omega_hdm = abun(n_fo,mass)                                                                           #Abundance of hdm
    plt.loglog(mass,omega_hdm,label = r'$factor = %s $'%i)                              #plot the given case in loglogscale
                                                                                      
plt.ylabel(r'$\Omega_{HDM}$ ', fontsize=18)
plt.xlabel(r'Mass [$eV$]', fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'$\Omega_{HDM}$ for Hot Dark Matter', fontsize=20)
plt.grid(True)

plt.legend(prop=dict(size=18))
plt.show()

### Cold Dark Matter

The cold dark matter (CDM) case is very similar to the HDM case (see above), except that the particle density is now given by a different expression: 
$n_{CDM}(T) = g^\star\left( \frac{m T}{2 \pi} \right)^{3/2} e^{-m/T}$. 

Now solving the Hubble equation for $T_{fo}$ cannot be done analytically, so we solve it numerically. For a non relativistic particle we take $\frac{1}{3}c n(T) \sigma = \sqrt{\frac{8 \pi G}{3}}\rho^{1/2}.$ Same as before, we take the radiation dominated energy density and model the cross section as $\sigma = G_F^2 T^2$. Now we will express $T_{fo}$ in terms of the dimensionless quantity $x \equiv \frac{m}{T}$. We expect $x$ to be linearly proportional to the mass $m$. We make a plot of this to cross check our results, which can be seen below.


Now that we have $T_{fo}$, we fill this into $n(T_{fo})$, redshift this to $n(T_0)$ and finally fill into $\Omega_{CDM} = \frac{m n(T_0)}{\rho_c}$. Again, we plot this abundance as a function of the mass, for different values of the cross section.


In [ ]:
#Calculate the relation between x and mass
def func3(T,mass,G_f):
    """Function needed to solve the Hubble equation, numerically and find T_fo
    Input: 3 single values, output: single value"""
    func3 = np.sqrt(g_s_T0) * G_f**2  *np.exp(-mass/T) - np.sqrt(32*G/5)*(np.pi**3) *(mass*T)**(-3/2)
    return func3

def cal_Tcdm(mass,G_f):
    """Calculates T_fo for a given mass
    Input: 2 single values, output: single value"""
    T_cdm = root(func3,T_fo,args = (mass,G_f))
    return T_cdm

x_l = []
for i in mass:
    j = i * GeV                                                                                  #to have mass in GeV range
    T_cdm = cal_Tcdm(j,G_f)
    x_l.append(j/T_cdm['x'])                                                                       #saving the calculated x

plt.figure(1,figsize=(14, 6))                              #Plotting x at freeze out as a function of mass on semilog scale
plt.semilogx(mass, x_l)
plt.xlabel(r'$Mass$ [$GeV$] ',fontsize=18)
plt.ylabel(r'x',fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'$T_{fo}$ relations to mass',fontsize=20)
plt.grid(True)

# plt.legend(prop=dict(size=18))
plt.show()

In [ ]:
def nd_cdm(m,T,g):
    """Calculates the number density
    Input: 3 single values, output: single value"""
    n_cdm = g*(m*T/(2*np.pi))**(3/2) *np.exp(-m/T)
    return n_cdm

def ab_cdm(m,n,T_fo):
    """Calculates the abundance
    Input: 3 single values, output: single value"""
    n_0 = n*(g_s_T0 * T_CMB**3)/(g_s_Tfo * T_fo**3)
    return m*n_0/rho_c

### Intermezzo - Extracting $g_s^\star$

In the cells below we extract and interpolate the degrees of freedom from the plasma. Since different particles decouple at different temperatures of the plasma, $g_s^\star$ can be seen as a function dependent on the temperature $T$. Here we determine this function and interpolate it for different values of $T$.

In [ ]:
#Shows the degrees of freedom as function which we use
#it remains constant outside the range of the data
from numpy import genfromtxt
from scipy.interpolate import interp1d
from scipy.integrate import odeint

my_data = genfromtxt('data.csv', delimiter=',')
x       = my_data[:,0]
y       = my_data[:,1]
g       = interp1d(x,y,fill_value="extrapolate")
T       = np.logspace(-3,7,1.0e7)

plt.figure(1,figsize=(14, 6))
plt.semilogx(T,g(T),label = 'Interpolated function')
plt.semilogx(x,y,label = "Analytical")
plt.ylabel(r'$g^\star$',fontsize=18)
plt.xlabel(r'$T$ [$MeV$]',fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'$g^\star$ as function of T ', fontsize=20)
plt.legend(prop=dict(size=18))
plt.grid(True)

In [ ]:
#Plots the cold dark matter abundance as function of mass
#For different cross section and shows the difference between 
#assuming constant degrees of freedom and as function of T
plt.figure(1,figsize=(18, 12))
mass   = np.linspace(1.0,1.0e2,500)
factor = np.logspace(-1,2,4)

for k in factor:
    omega_l = [] #List which uses constant g^\star
    omega_c = [] #List which uses variable g^\star
    
    for i in mass:
        j = i * GeV
        T_cdm = cal_Tcdm(j,k*G_f)['x'][0]
        
        #calculate the cdm abundance for constant g^\star
        n_cdm     = nd_cdm(j,T_cdm,g_dof) #Number density
        omega_cdm = ab_cdm(j,n_cdm,T_cdm) #Abundance
        omega_l.append(omega_cdm)

        #calculate the cdm abundance for g^\star as function of T
        n_cc = nd_cdm(j,T_cdm,g(T_cdm/MeV)) #Number density
        omega_cc = ab_cdm(j,n_cc,T_cdm) #Abundance
        omega_c.append(omega_cc)

    plt.loglog(mass, omega_l,label = '$g^\star = constant, factor = %s$' %k)
    plt.loglog(mass, omega_c,label = '$g^\star = f(T), factor = %s$' %k)

plt.xlabel(r'Mass [$GeV$]',fontsize=18)
plt.ylabel(r'$\Omega_{CDM}$',fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'$\Omega_{CDM}$ for Cold Dark Matter',fontsize=20)
plt.grid(True)

plt.legend(prop=dict(size=15))
plt.show()

## Part 2: Non-instantaneous freeze-out
We now have to solve the differential equation given in [Steigman et al.](https://arxiv.org/pdf/1204.3622.pdf):

$\frac{dY}{dx}= \frac{s \left<\sigma v\right>}{Hx}\left(1+\frac{1}{3}\frac{d(\ln g_s^\star)}{d(\ln T)}\right)\left(Y^2_{eq} - Y^2 \right)$. 

We also know
$Y_{eq}=\frac{45}{2 \pi^4} \left(\frac{\pi}{8}\right)^{1/2} \frac{g_{\chi}}{g_s^\star} x^{3/2} e^{-x}$, $s=\frac{2 \pi^2}{45} g_s^\star \left(\frac{m}{x}\right)^3$, $H = \sqrt{\frac{ 8 \pi G }{3} \cdot \frac{\pi^2 \rho}{30}} \left(\frac{m}{x}\right)^2$ and $x \equiv \frac{m}{T}$. If we now rewrite the above equation in terms of $x$ and not $T$, we get the following expression:

$\frac{dY}{dx}= \left<\sigma v\right> \frac{2 \pi^2}{45} g_s^\star \frac{m}{x^2} \sqrt{\frac{90}{8 \pi^3 G g_s^\star}} \left(1-\frac{m}{3}\frac{d(\ln g_s^\star)}{d x}\right)\left( \left( \frac{45}{2 \pi^4} \cdot \frac{g_{\chi}}{g_s^\star}\right)^2 \frac{\pi}{8} x^3 e^{-2x} - Y^2 \right)$. We now have to solve this differential equation and plot $Y$ as a function of $T$.

In order to do this, we must first make several assumptions. The first assumption is that the relativistic degrees of freedom of the plasma, $g_s^\star (T)$, can be treated as a step function (see intermezzo above). We also assume $ \left< \sigma v \right>$ is given by the equipartition theorem.

We have extracted the values for $g_s^\star$ from [Steigman et al.](https://arxiv.org/pdf/1204.3622.pdf). and interpolated them to obtain a function for $g_s^\star$.

In [ ]:
# In this cell we calculate the derivative of ln(g^\star) with respect to ln(T) 

# calculate a range for mass and x, where variable x is denoted as m_T
mass = np.logspace(-1,4,1.0e6) * GeV
m_T  = np.logspace(-5,10,1.0e6) 

# calculate the logarithmic derivative. Our function g^\star can be written as g(T)=g(mass/m_T) and
# the derivative d/dlog(x) is rewritten to x*(d/dx)
step_size = 0.0001
derivs    = -m_T*(np.log(g(mass/m_T - step_size*mass/m_T)) -np.log(g(mass/m_T)))/(step_size*m_T)

plt.figure(1,figsize=(14, 6))
plt.semilogx(mass/m_T,derivs)
plt.ylabel(r'$\frac{\partial g^\star}{\partial T}$',fontsize=18)
plt.xlabel(r'$T$ [$MeV$]',fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'derivative $g^\star$ as function of T ',fontsize=20)
plt.grid(True)

In [ ]:
#mass = np.logspace(-1,4,1.0e6)
mass    = 1 * GeV
m_T     = np.logspace(-3,3,1.0e6)
sigma_v = 1.0e-25 * cm**3 / s
Lambda  = (2.67e35) * mass * sigma_v


# define our derivative in terms of T as a function
logderiv = interp1d(mass/m_T,derivs,fill_value="extrapolate") 

def model(w,x):
    """ODE from Steigman"""
    
    # calculate weq with the approximation that we can use the non-rel. expression for n_eq (see Steigman) 
    weq = math.log(0.145*(1/g(x)) * (x**(3./2))*math.exp(-x))
    
    # this is equation 23 from Steigman, where we made the assumption that g_rho ~ 1
    dwdx = (Lambda/(x**2))*(1+(1/3)*logderiv(mass/x))*(np.exp(2*weq-w)-np.exp(w))*g(mass/x)
    return dwdx

# initial condition, for now a random guess
w0 = 1e-3

# time points
x = np.linspace(1e-5,1.0e-4,100)

# solve ODE
w = odeint(model,w0,x,mxstep=6000000) #mstep to avoid a warning

# plot Y vs x (using Y = exp(w))
plt.figure(1,figsize=(14, 6))
plt.plot(x,np.exp(w))
plt.ylabel(r'$Y$',fontsize=18)
plt.xlabel(r'$x$',fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=20)
plt.title(r'Y as function of x ',fontsize=20)
plt.grid(True)
plt.show()